# Predicting Readability of Scientific Research Paper Abstracts

## By: Srihari Raman

### About the Project

Ever wondered how to make your life easier while reading research papers? Well, step 1 is to know whether or not the paper will be readable! In today's day and age, reading things is tedious, especially when it comes to research papers. Some are worded with heavy jargon, whereas the others are simply not interesting enough. In an effort to build an NLP model to assist with that issue, I evaluate some of the various models that are available in the ML world today.

**For a detailed outline, please see the Project Proposal!**

## Project Setup

In [43]:
# !pip install textstat
# !pip install asyncio
# !pip install aiohttp
# !pip install tqdm
# ! pip install xgboost

In [44]:
# Imports
import os
import utility.arxiv_utility as axu
import utility.preprocess_utility as ppu
import asyncio
import aiohttp
from tqdm import tqdm_notebook
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import polars as pl

from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import numpy as np
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

from transformers import BertTokenizer
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.layers import Dense
from transformers import TFDistilBertModel, DistilBertTokenizer

nltk.download('stopwords')
nltk.download('punkt')

%autoawait asyncio

### Building the Development Dataset

#### Note:

To ensure randomness in the lexical levels of the abstracts collected, the queries are limited to broad topics related to 4 main subject matters—science, english, math, and humanities. I understand that these subjects may naturally exclude other subject matters from consideration, but for the general purpose of this project, I take those considerations into account.

In [45]:
# Main function to create df, run asynch --> need to create this in order to run asynch with Jupyter
async def data_collection():
    query = ["science", "english", "math", "humanities"]
    queries_list, abstracts = await axu.get_abstracts(query, max_results=2000)
    df = axu.batch_add_to_df(abstracts, queries_list)
    return df

In [46]:
# Load/create training data
if os.path.exists("./data/arxiv_fulldata.csv"):
    print("Training dataframe found.")
    df = pd.read_csv("./data/arxiv_fulldata.csv")

else:
    print("Creating training data.")
    df = await data_collection()
    df.to_csv("./data/arxiv_fulldata.csv")

### EDA - Development Set

In [47]:
df.head()

In [48]:
df.dtypes

In [49]:
df.columns

In [89]:
# Check for class imbalance in training data and the average readability scores
avg_readability = df.groupby('query')['flesch_reading_ease'].mean().reset_index()
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar plot for average readability scores
sns.barplot(ax=axes[0], x='query', y='flesch_reading_ease', data=avg_readability, palette='viridis')
axes[0].set_title('Average Readability Score by Query')
axes[0].set_xlabel('Query')
axes[0].set_ylabel('Average Flesch Reading Ease Score')
# axes[0].tick_params(axis='x', rotation=45)

# Count plot for the number of abstracts per query
sns.countplot(ax=axes[1], x='query', data=df, palette='viridis')
axes[1].set_title('Count of Abstracts by Query')
axes[1].set_xlabel('Query')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.savefig("eda_dev.png")
plt.show()

In [90]:
# Check for correlations between numerical columns
numerical_columns = ['num_words', 'num_sentences', 'num_characters', 'flesch_reading_ease']
df_numerical = df[numerical_columns]
correlation_matrix = df_numerical.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.savefig("heatmap_corr.png")
plt.show()

### Building the Testing Dataset

In [52]:
test_queries = [
    'astrophysics', 'neuroscience',
    'topology', 'cryptography',
    'postmodern literature', 'sociolinguistics',
    'cultural anthropology', 'philosophy'
]

In [96]:
test_df.shape

In [53]:
async def test_data_collection():
    queries_list, abstracts = await axu.get_abstracts(test_queries, max_results=500)
    test_df = axu.batch_add_to_df(abstracts, queries_list)
    return test_df

In [54]:
# Load/create training data
if os.path.exists("./data/arxiv_test.csv"):
    print("Test dataframe found.")
    test_df = pd.read_csv("./data/arxiv_test.csv")

else:
    print("Creating training data.")
    test_df = await test_data_collection()
    test_df.to_csv("./data/arxiv_test.csv")

In [55]:
test_df.head()

### EDA - Training Set

In [91]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar plot for average readability scores
sns.barplot(ax=axes[0], x='query', y='flesch_reading_ease', data=avg_readability, palette='viridis')
axes[0].set_title('Average Readability Score by Query')
axes[0].set_xlabel('Query')
axes[0].set_ylabel('Average Flesch Reading Ease Score')
# axes[0].tick_params(axis='x', rotation=45)

# Count plot for the number of abstracts per query
sns.countplot(ax=axes[1], x='query', data=test_df, palette='viridis')
axes[1].set_title('Count of Abstracts by Query')
axes[1].set_xlabel('Query')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.savefig("eda_test.png")
plt.show()

### Feature Engineering - Development & Training Sets

In the feature engineering process, Polars (a faster development to Pandas) was used for efficient processing. Polars utilizes lazy execution and parallelization, which allows for all the cores in the CPU to be utilized for faster data processing.

I wrote an accessory script imported as 'ppu' that defines all the preprocessing steps. However, I've also outlined them below:

- Lower case folding
- Work tokenization utilizing NLTK
- Stop word removal using NLTK stopwords
- Generating Word Embeddings with Gensim Word2Vec

**See the 'preprocessing_utility.py' file for additional information!**

In [57]:
%%time
df = ppu.preprocess_dataframe(df)

In [58]:
%%time
test_df = ppu.preprocess_dataframe(test_df)

## Model Development

In this section, I develop 3 different types of models to predict the Flesch Reading Ease score, which is a metric designed to calculate how difficult it would be to read an abstract. Both the linear regression and the Gradient Boosted Random Forest Regressor models take in the number of words, number of sentences, number of characters, and word embeddings as parameters. The fine-tuned BERT model, however, only takes in the embeddings. This is was intentionally done to test the prediction power of the state-of-the-art DistilBERT model compared to a the traditional regressor models.

In [59]:
df.head()

In [60]:
test_df.head()

### Baseline Model - Linear Regression with Embeddings

The baseline model used for comparative analysis is a linear regression model that aims to predict the readability score of the abstracts.

In [61]:
# Extract the relevant numerical columns and the target column
numerical_columns = ['num_words', 'num_sentences', 'num_characters']
target_column = 'flesch_reading_ease'

# Extract numerical features and target column
X_numerical = df[numerical_columns]
y = df[target_column]

In [62]:
# Extract embeddings and flatten into DataFrame columns --> given by ChatGPT
embeddings = np.array(df['embedding'].tolist())
embedding_df = pd.DataFrame(embeddings)

X = pd.concat([X_numerical.reset_index(drop=True), embedding_df.reset_index(drop=True)], axis=1)
X.columns = X.columns.astype(str)

In [63]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
# Initialize and train the linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predict the target on the test set
y_pred = lr_model.predict(X_test)

In [65]:
# Evaluate on test split just for performance bench --> to be evaluated on actual test set later
mse_lg = mean_squared_error(y_test, y_pred)
mae_lg = mean_absolute_error(y_test, y_pred)
r2_lg = r2_score(y_test, y_pred)

print(f'Baseline Linear Regression Model Performance:')
print(f'Mean Squared Error: {mse_lg}')
print(f'Mean Absolute Error: {mae_lg}')
print(f'R-squared: {r2_lg}')

### Gradient Boosted Random Forest Regressor (XGBoost)

In [66]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

In [67]:
# Train XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_model.fit(X_train, y_train)

In [68]:
# Predict on same split as LR model
y_pred_xgb = xgb_model.predict(X_test)

In [69]:
# Evaluate on same test split as LR model
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f'Advanced XGBoost RFR Model Performance: \n')
print(f'Mean Squared Error: {mse_xgb}')
print(f'Mean Absolute Error: {mae_xgb}')
print(f'R-squared: {r2_xgb}')

### Pre-trained DistilBERT Model With Regression NN Layer

In [70]:
# Tokenize abstracts
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
def tokenize_function(text):
  """Creates a tokenizer function for each text, to be applied to the DF.

    Args:
      text (str): The text to create the tokenizer for

    Returns:
      tokenizer (ts.BertTokenizer): A Bert tokenizer object
    """
  return tokenizer(text, padding='max_length', truncation=True, max_length=512)

# Apply tokenizer to the dataset
df['inputs'] = df['abstract'].apply(tokenize_function)

In [72]:
def create_dataset(df):
    """
    Creates a tensor flow dataset to be passed to the BERT model

    Args:
    dataframe (pd.Dataframe): The dataframe to convert

    Returns:
    tf.data.Dataset.from_tensor_slices: TF dataset with tensor slices for easy training
    """
    input_ids = []
    attention_masks = []

    for inputs in df['inputs']:
        input_ids.append(inputs['input_ids'])
        attention_masks.append(inputs['attention_mask'])

    inputs = {
        'input_ids': tf.constant(input_ids),
        'attention_mask': tf.constant(attention_masks)
    }
    labels = tf.constant(df['flesch_reading_ease'].values)
    return tf.data.Dataset.from_tensor_slices((inputs, labels))

In [73]:
class DistilBertRegressor(tf.keras.Model):
    def __init__(self, distilbert_model_name):
        super(DistilBertRegressor, self).__init__()
        self.distilbert = TFDistilBertModel.from_pretrained(distilbert_model_name)
        self.regressor = tf.keras.layers.Dense(1, activation='linear')

    # This is the function (defined by TF for NNs) that uses forward pass to get the prediction outputs
    def call(self, inputs):
        # Gets the BERT last layer output after passing in tokens
        distilbert_output = self.distilbert(inputs)

        # This gets hidden state values of the CLS token, which represents the entire sequence
        hidden_state = distilbert_output.last_hidden_state[:, 0, :]

        # Return the regression output after passing CLS token inputs through Dense NN
        return self.regressor(hidden_state)

In [74]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

In [75]:
# Define the R2 score class to get r2 score for each epoch --> GPT assisted me with this
class R2Score(tf.keras.metrics.Metric):
    def __init__(self, name='r2_score', **kwargs):
        super(R2Score, self).__init__(name=name, **kwargs)
        self.sse = self.add_weight(name='sse', initializer='zeros')
        self.sst = self.add_weight(name='sst', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        self.sse.assign_add(tf.reduce_sum(tf.square(y_true - y_pred)))
        self.sst.assign_add(tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true))))

    def result(self):
        return 1 - self.sse / (self.sst + tf.keras.backend.epsilon())

    def reset_state(self):
        self.sse.assign(0.0)
        self.sst.assign(0.0)

In [76]:
# Avoiding training if weights file exists, else train the model
if os.path.exists('model_weights.h5'):
    print("Model weights file found.")

    # To avoid subclass error, needed to forward pass dummy data
    bert_model = DistilBertRegressor('distilbert-base-uncased')
    dummy_inputs = tokenizer(["This is a dummy sentence."], return_tensors='tf', padding=True, truncation=True, max_length=128)

    # Forward pass the dummy data
    bert_model(dummy_inputs)  # Forward pass to create the variables
    bert_model.load_weights('model_weights.h5')
    
    # Load the result dictionary also if it exists
    if os.path.exists('result_dict.pkl'):
        with open('result_dict.pkl', 'rb') as f:
            result_dict = pickle.load(f)
        print("Result dictionary loaded successfully.")

else:
    print("Model weights file not found. Training the model.")

    # Convert the datasets to TensorFlow datasets and create train and val datasets
    BATCH_SIZE = 8
    train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(100)
    val_dataset = val_dataset.batch(BATCH_SIZE)
    
    bert_model = DistilBertRegressor('distilbert-base-uncased')
    bert_model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.MeanSquaredError(), R2Score()])    # Train the model on the small subset
    results = bert_model.fit(train_dataset, validation_data=val_dataset, epochs=3)
    
    # Save the model's weights
    bert_model.save_weights('model_weights.h5')
    
    # Save the training history result_dict
    result_dict = results.history
    with open('result_dict.pkl', 'wb') as f:
        pickle.dump(result_dict, f)
        print("Results dictionary serialized.")

In [77]:
print("Validation Loss:", result_dict['loss'])
print("Validation Mean Absolute Error (MAE):", result_dict['mean_absolute_error'])
print("Validation Mean Squared Error (MSE):", result_dict['mean_squared_error'])
print("Validation R-squared:", result_dict['r2_score'])

In [92]:
# Explore the per-epoch statistics in 2x2 plot for MAE, Loss, MSE, and R2
epochs = range(1, len(result_dict['val_loss']) + 1)
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Validation Loss
sns.lineplot(ax=axes[0, 0], x=epochs, y=result_dict['val_loss'], marker='o')
axes[0, 0].set_title('Validation Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')

# Validation MAE
sns.lineplot(ax=axes[0, 1], x=epochs, y=result_dict['val_mean_absolute_error'], marker='o')
axes[0, 1].set_title('Validation MAE')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('MAE')

# Validation MSE
sns.lineplot(ax=axes[1, 0], x=epochs, y=result_dict['val_mean_squared_error'], marker='o')
axes[1, 0].set_title('Validation MSE')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('MSE')

# Validation R-squared
sns.lineplot(ax=axes[1, 1], x=epochs, y=result_dict['val_r2_score'], marker='o')
axes[1, 1].set_title('Validation R-squared')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('R-squared')

plt.tight_layout()
plt.savefig("bert_per_epoch_fig.png")
plt.show()

## Model Analysis

### Run Models on Testing Data

#### Baseline LR Model

In [82]:
X_test_embeddings = np.vstack(test_df['embedding'].values)
X_test_other_features = test_df[['num_words', 'num_sentences', 'num_characters']].values

# Concatenate embeddings and other features
X_test = np.hstack((X_test_embeddings, X_test_other_features))
y_true = test_df['flesch_reading_ease'].values

In [83]:
# Make predictions
linear_predictions = lr_model.predict(X_test)

In [84]:
# Linear Regression model evaluation
mse_linear = mean_squared_error(y_true, linear_predictions)
mae_linear = mean_absolute_error(y_true, linear_predictions)
r2_linear = r2_score(y_true, linear_predictions)

print(f'Baseline Linear Regression Model Performance: \n')
print(f'Mean Squared Error: {mse_linear}')
print(f'Mean Absolute Error: {mae_linear}')
print(f'R-squared: {r2_linear}')

#### XGB Model

In [85]:
xgb_predictions = xgb_model.predict(X_test)

In [86]:
# XGB model evaluation
mse_xgb = mean_squared_error(y_true, xgb_predictions)
mae_xgb = mean_absolute_error(y_true, xgb_predictions)
r2_xgb = r2_score(y_true, xgb_predictions)

print(f'Advanced XGBoost RFR Model Performance: \n')
print(f'Mean Squared Error: {mse_xgb}')
print(f'Mean Absolute Error: {mae_xgb}')
print(f'R-squared: {r2_xgb}')

#### Fine-Tuned BERT Model

In [87]:
# Predict with BERT --> Takes 4-9 minutes
inputs = tokenizer(test_df['abstract'].tolist(), return_tensors='tf', padding=True, truncation=True, max_length=128)
distilbert_predictions = bert_model.predict(dict(inputs)).flatten()

In [88]:
# BERT model evaluation
mse_bert = mean_squared_error(y_true, distilbert_predictions)
mae_bert = mean_absolute_error(y_true, distilbert_predictions)
r2_bert = r2_score(y_true, distilbert_predictions)

print(f'Fine-tuned DistilBERT Model Performance: \n')
print(f'Mean Squared Error: {mse_bert}')
print(f'Mean Absolute Error: {mae_bert}')
print(f'R-squared: {r2_bert}')

#### Save Performance Data

In [79]:
# Load/create metrics dataframe for offline use
if os.path.exists("./data/model_evaluation.csv"):
    print("Metrics dataframe found.")
    metrics_df = pd.read_csv("./data/model_evaluation.csv")

else:
    print("Storing metrics data.")

    metrics_df = pd.DataFrame({
        'Model': ['DistilBERT Regressor', 'Linear Regression', 'Random Forest'],
        'MSE': [mse_bert, mse_linear, mse_xgb],
        'MAE': [mae_bert, mae_linear, mae_xgb],
        'R-squared': [r2_bert, r2_linear, r2_xgb]
    })

    metrics_df.to_csv("./data/model_evaluation.csv")

In [99]:
metrics_df

### Model Comparisons + Analysis

In [93]:
# Calculate and plot residuals
residuals_distilbert = y_true - distilbert_predictions
residuals_linear = y_true - linear_predictions
residuals_xgb = y_true - xgb_predictions

In [94]:
# Residual plot
fig, axes = plt.subplots(2, 2, figsize=(15, 15))

# DistilBERT Regressor
sns.scatterplot(x=distilbert_predictions, y=residuals_distilbert, ax=axes[0, 0])
axes[0, 0].set_title('DistilBERT Regressor Residuals')
axes[0, 0].set_xlabel('Predicted Values')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].axhline(0, color='r', linestyle='--')

# Linear Regression
sns.scatterplot(x=linear_predictions, y=residuals_linear, ax=axes[0, 1])
axes[0, 1].set_title('Linear Regression Residuals')
axes[0, 1].set_xlabel('Predicted Values')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].axhline(0, color='r', linestyle='--')

# Random Forest
sns.scatterplot(x=xgb_predictions, y=residuals_xgb, ax=axes[1, 0])
axes[1, 0].set_title('Random Forest Residuals')
axes[1, 0].set_xlabel('Predicted Values')
axes[1, 0].set_ylabel('Residuals')
axes[1, 0].axhline(0, color='r', linestyle='--')

axes[1, 1].axis('off')

plt.tight_layout()
plt.savefig("residual_plots.png")
plt.show()

In [95]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plotting Mean Squared Error (MSE)
sns.barplot(x='Model', y='MSE', data=metrics_df, ax=axes[0])
axes[0].set_title('Logarithmic Mean Squared Error (MSE) Comparison')
axes[0].set_ylabel('Log MSE')
axes[0].set_yscale('log')

# Adding values above bars --> given by GPT
for p in axes[0].patches:
    axes[0].annotate(format(p.get_height(), '.2e'), 
                        (p.get_x() + p.get_width() / 2., p.get_height()), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 10), 
                         textcoords = 'offset points')

# Plotting Mean Absolute Error (MAE)
sns.barplot(x='Model', y='MAE', data=metrics_df, ax=axes[1])
axes[1].set_title('Logarithmic Mean Absolute Error (MAE) Comparison')
axes[1].set_ylabel('Log MAE')
axes[1].set_yscale('log')

# Adding values above bars --> given by GPT
for p in axes[1].patches:
    axes[1].annotate(format(p.get_height(), '.2e'), 
                        (p.get_x() + p.get_width() / 2., p.get_height()), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 10), 
                         textcoords = 'offset points')

plt.tight_layout()
plt.savefig("performance.png")
plt.show()

### Acknowledgments

*This project was completed as part of the CS4120 Natural Language Processing class at Northeastern University.*

*A sincere thanks to ArXiv's open-source research forum for providing the abstracts as data for this project.*

As a part of the development of this project, ChatGPT was used extensively to assist with error debugging, explanation of BERT tokenizers and sub-model creation, as well as visualization development.